In [1]:
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib

import os
import sys

#import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import torchvision
import torchvision.transforms as transforms

matplotlib.rcParams['figure.figsize'] = (20,10)

In [51]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print(torch.version.cuda)
print(device)
#Ikke i brug endnu

9.0.176
cuda:0


## Kode til at gemme modeltilstande

In [4]:
def SaveState(model):
    x = datetime.datetime.now()
    filename = '../checkpoints/model_state_%s_%.4f.pt' % (x.strftime("%Y%m%d-%H%M"), acc)
    torch.save(model.state_dict(), filename)
    print('Model saved as:\n%s' % os.path.abspath(filename))
    
def LoadState(filename):
    model = torch.load(filename)
    model.eval()
    return model

## Indlæs data
Det antages at dataen ligger i `'../data'`

In [5]:
data_raw = np.load('../data/cullpdb+profile_5926.npy')

## Omform data

In [59]:
data = data_raw.reshape((-1,700,57))

amino_acid_recidues  = data[:,:,:22]
amino_seq_profiles   = data[:,:,35:]
sec_structure_labels = data[:,:,22:31]
solvent_access       = data[:,:,33:35]

ext_x = np.concatenate((amino_acid_recidues, amino_seq_profiles), axis=2)

x = ext_x
#x = amino_acid_recidues
#y = sec_structure_labels
y = solvent_access

input_channels  = x.shape[2]
output_channels = y.shape[2]

print('Kanaler:\nInput:  %d\nOutput: %d' % (input_channels, output_channels))

print('Fuldt datasæt shape:')
print('X: ', x.shape)
print('Y: ', y.shape)

y_train_unrot = y[:5430]
y_test_unrot = y[5435:5690]
y_validation_unrot = y[5690:5926]

#x = x.reshape(-1,22,700)
#y = y.reshape(-1,9,700)
x = np.rot90(x, axes=(1,2))
y = np.rot90(y, axes=(1,2))

x = np.flip(x, 1)
y = np.flip(y, 1)

print('Fuldt datasæt vendt shape:')
print('X: ', x.shape)
print('Y: ', y.shape)

x_train = x[:5430]
y_train = y[:5430]

x_test = x[5435:5690]
y_test = y[5435:5690]

x_validation = x[5690:5926]
y_validation = y[5690:5926]

print('Splittet ud i training og testing:')
print('(Train) X: ', x_train.shape)
print('(Train) Y: ', y_train.shape)
print('(Test)  X: ', x_test.shape)
print('(Test)  Y: ', y_test.shape)
print('(Validation)  X: ', x_validation.shape)
print('(Validation)  Y: ', y_validation.shape)

torch_X_train = torch.from_numpy(x_train).type(torch.FloatTensor).to(device)
torch_Y_train = torch.from_numpy(y_train).type(torch.FloatTensor).to(device)
torch_X_test  = torch.from_numpy(x_test).type(torch.FloatTensor).to(device)
torch_Y_test  = torch.from_numpy(y_test).type(torch.FloatTensor).to(device)
torch_X_validation  = torch.from_numpy(x_validation).type(torch.FloatTensor).to(device)
torch_Y_validation  = torch.from_numpy(y_validation).type(torch.FloatTensor).to(device)

Kanaler:
Input:  44
Output: 2
Fuldt datasæt shape:
X:  (5926, 700, 44)
Y:  (5926, 700, 2)
Fuldt datasæt vendt shape:
X:  (5926, 44, 700)
Y:  (5926, 2, 700)
Splittet ud i training og testing:
(Train) X:  (5430, 44, 700)
(Train) Y:  (5430, 2, 700)
(Test)  X:  (255, 44, 700)
(Test)  Y:  (255, 2, 700)
(Validation)  X:  (236, 44, 700)
(Validation)  Y:  (236, 2, 700)


## Sæt data sammen i DataLoader

In [60]:
BATCH_SIZE = 50 # 250 har virket godt før

train = torch.utils.data.TensorDataset(torch_X_train, torch_Y_train)
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=BATCH_SIZE, shuffle=False)

## Definér modellen

In [61]:
layer_width = 80
kernel_sizes = [5, 11, 5, 5]

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=input_channels,       
                out_channels=layer_width,      
                kernel_size=kernel_sizes[0],        
                stride=1,             
                padding=int(kernel_sizes[0]/2),            
            ),                        
            nn.ReLU(),                
        )
        self.conv2 = nn.Sequential(   
            nn.Conv1d(
                in_channels=layer_width,       
                out_channels=layer_width,      
                kernel_size=kernel_sizes[1],        
                stride=1,             
                padding=int(kernel_sizes[1]/2),            
            ),                        
            nn.ReLU(),                
        )
        self.conv3 = nn.Sequential(   
            nn.Conv1d(
                in_channels=layer_width,       
                out_channels=layer_width,      
                kernel_size=kernel_sizes[2],        
                stride=1,             
                padding=int(kernel_sizes[2]/2),            
            ),                        
            nn.ReLU(),                
        )
        self.out = nn.Sequential(     
            nn.Conv1d(
                in_channels=layer_width,       
                out_channels=output_channels,       
                kernel_size=kernel_sizes[3],        
                stride=1,             
                padding=int(kernel_sizes[3]/2),            
            ),
            nn.ReLU(),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        output = self.out(x)
        return output #, x    # return x for visualization

## Instantiér modellen

In [62]:
cnn = CNN().to(device)
print(cnn)
print('Network is on device: "%s"' % device)
# Hyperparametre
LR = 0.005              # learning rate

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.BCELoss()  #Binary Cross Entropy Loss
validation_loss_func = nn.BCELoss()
test_loss_func = nn.BCELoss()

CNN(
  (conv1): Sequential(
    (0): Conv1d(44, 80, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv1d(80, 80, kernel_size=(11,), stride=(1,), padding=(5,))
    (1): ReLU()
  )
  (conv3): Sequential(
    (0): Conv1d(80, 80, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
  )
  (out): Sequential(
    (0): Conv1d(80, 2, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): Softmax()
  )
)
Network is on device: "cuda:0"


In [22]:
# Til visualisering senere
losses     = []
accuracies = []
steps      = []
steps_cum  = []
epochs     = []

## Funktion til at måle accuracy

In [158]:
def CalculateAccuracyOld(calc_values, real_values):
    relevant = 0
    calc_values = calc_values.detach().numpy()              # Omform til numpy
    calc_values = np.flip(calc_values, 1)                   # Omgør spejlning og 
    calc_values = np.rot90(calc_values, k=-1, axes=(1,2))   # rotation
    calc_values_oh = np.zeros(calc_values.shape)            # Lav et nyt tomt array med samme shape
    for prot_nr, protein in enumerate(calc_values):         # For hvert protein
        for amino_nr, amino in enumerate(protein):          # For hver aminosyre
            feature = np.argmax(amino)                      # Find den højeste sandsynlighed
            if real_values[prot_nr, amino_nr, feature] != 1:
                relevant += 1
            calc_values_oh[prot_nr, amino_nr, feature] = 1  # Sæt den til 1
    differences = abs(calc_values_oh.flatten() - real_values.flatten())
    print(relevant)
    false = int(sum(differences))
    #total = relevant
    total = len(real_values.flatten())
    #print(total_old)
    true  = total-false
    return (true/total)

In [19]:
def CalculateAccuracy(calc_values, real_values):
    NoSeq = 8
    real_labels = np.argmax(real_values, axis=2)            # Kollaps one-hot til rene labels
    real_mask = real_labels == NoSeq                        # Lav maske af dem der er NoSeq
    calc_values = calc_values.cpu().detach().numpy()              # Omform til numpy
    calc_values = np.flip(calc_values, 1)                   # Omgør spejlning og 
    calc_values = np.rot90(calc_values, k=-1, axes=(1,2))   # rotation
    calc_labels = np.argmax(calc_values, axis=2)            # Kollaps one-hot til rene labels
    correct = calc_labels == real_labels                    # Find hvilke forudsigelser der er korrekte
    correct_masked = np.ma.masked_array(correct, real_mask) # Filtrér dem er er NoSeq
    theMean = np.mean(correct_masked)                       # Tag gennemsnittet af sættet
    return theMean

## Træn modellen

In [63]:
EPOCH = 5               # train the training data n times

step_cum = -1
for epoch in range(EPOCH):
    print('\nEpoch: ', epoch+1)
    for step, (b_x, b_y) in enumerate(train_loader):
        step_cum += 1
        output = cnn(b_x)#[0]               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 10 == 0:
            test_output = cnn(torch_X_validation)
            vloss = validation_loss_func(test_output, torch_Y_validation)
            acc = CalculateAccuracy(test_output, y_validation_unrot)
            sys.stdout.write('\rStep [%d]:\tTrain:[Loss: %.5f] \t Validation:[Loss: %.5f,\tAccuracy: %.4f%%]' % (step, loss.item(),  vloss.item(), acc*100))
            #epochs.append(epoch)
            #losses.append(vloss.item())
            #accuracies.append(acc)
            #steps.append(step)
            #steps_cum.append(step_cum)
            
print('\nDone training.')


Epoch:  1
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  2
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  3
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  4
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  5
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  6
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  7
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  8
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  9
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Epoch:  10
Step [100]:	Train:[Loss: 0.69315] 	 Validation:[Loss: 0.69315,	Accuracy: 100.0000%]
Done training.


## Endelig test af accuracy

In [24]:
print('Running on the test set:')
test_output = cnn(torch_X_test)
vloss = test_loss_func(test_output, torch_Y_test)
acc = CalculateAccuracy(test_output, y_test_unrot)
sys.stdout.write('Loss: %.5f,\tAccuracy: %.4f%%' % (vloss.item(), acc*100))

Running on the test set:
Loss: 0.05517,	Accuracy: 65.2500%

In [149]:
SaveState(cnn)

Model saved as:
/home/simonsen/Documents/Uni/bachelor/git/Bachelor19/checkpoints/model_state_20190527-1433_0.9631.pt
